### Instalación de librerías

In [138]:
from IPython import display
%pip install gensim
%pip install nltk
display.clear_output()

### Imports

In [145]:
import nltk

from typing import List
from pathlib import Path
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize
from gensim.utils import simple_preprocess

In [140]:
nltk.download('punkt_tab', download_dir='./nltk_data')

[nltk_data] Downloading package punkt_tab to ./nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

### Preprocesamiento del corpus

Las siguientes funciones se encargan de preparar los textos de los siete libros de Harry Potter para el entrenamiento de modelos de embeddings con Word2Vec.

El procesamiento de la función process_books_for_word2vec() sigue los siguientes pasos:

1. Lectura de los libros: 
   Se leen todos los archivos .txt ubicados en la carpeta hp/. Cada archivo representa un libro completo.

2. Segmentación en oraciones:  
   Se utiliza la función sent_tokenize() de la librería nltk para dividir cada texto en oraciones.

3. Tokenización de oraciones:  
   Cada oración se tokeniza mediante simple_preprocess de Gensim, que:
   - Convierte todas las palabras a minúsculas,
   - Convierte todas las palabras a minúsculas,
   - Elimina signos de puntuación,
   - Elimina palabras con menos de 2 caracteres,
   - Opcionalmente elimina acentos (deacc=True).

El resultado final es una lista de listas de tokens (oraciones tokenizadas) que puede utilizarse directamente como entrada para entrenar modelos de embeddings con gensim.models.Word2Vec.

In [141]:
def read_all_books(directory: Path) -> List[str]:
    """
    Lee todos los archivos .txt dentro del directorio especificado.

    :param directory: Ruta a la carpeta con los archivos de texto.
    :return: Lista de strings, uno por cada libro completo.
    """
    all_text = []
    for book_file in sorted(directory.glob("*.txt")):
        with open(book_file, 'r', encoding='utf-8') as f:
            text = f.read()
            all_text.append(text)
    return all_text

def split_into_sentences(text: str) -> List[str]:
    """
    Divide un texto en oraciones usando la segmentación de NLTK.

    :param text: Texto unificado.
    :return: Lista de oraciones.
    """
    return sent_tokenize(text)

def tokenize_sentences(sentences: List[str]) -> List[List[str]]:
    """
    Tokeniza cada oración en palabras, eliminando puntuación y pasando a minúsculas.

    :param sentences: Lista de oraciones.
    :return: Lista de listas de palabras (tokens).
    """
    return [simple_preprocess(sentence, deacc=True) for sentence in sentences]

def process_books_for_word2vec(directory: Path) -> List[List[str]]:
    """
    Procesa todos los libros en un directorio para generar oraciones tokenizadas
    aptas para entrenamiento con Word2Vec.

    :param directory: Ruta a la carpeta con los libros.
    :return: Lista de oraciones tokenizadas.
    """
    raw_books = read_all_books(directory)
    all_sentences = []
    for book in raw_books:
        sentences = split_into_sentences(book)
        tokenized = tokenize_sentences(sentences)
        all_sentences.extend(tokenized)
    return all_sentences

In [155]:
BOOKS_DIR = Path("hp")

sentences = process_books_for_word2vec(BOOKS_DIR)

print(f"📄 Total de oraciones (docs): {len(sentences)}")

📄 Total de oraciones (docs): 66129


### Entrenamiento del modelo Word2Vec

In [146]:
w2v_model = Word2Vec(
    sentences=sentences,        # Lista de oraciones tokenizadas
    vector_size=100,            # Dimensión de los vectores (puede ajustarse)
    window=5,                   # Contexto de palabras (5 a la izquierda y 5 a la derecha)
    min_count=5,                # Palabras con frecuencia < 5 son ignoradas
    workers=4,                  # Número de hilos de procesamiento paralelo
    sg=1,                       # Skip-gram (1) en lugar de CBOW (0)
    epochs=10                   # Número de pasadas completas sobre el corpus
)

w2v_model.save("hp_word2vec.model")

### Pruebas

#### Similitud semántica

Se realiza una exploración del espacio de embeddings entrenado con Word2Vec usando el método:

```python
w2v_model.wv.most_similar('palabra_objetivo')
```

Este devuelve las palabras que tienen mayor similitud coseno con el vector asociado a la palabra dada. Es decir, aquellas que aparecen en contextos similares y, por lo tanto, tienen un significado o función similar dentro del corpus.

In [159]:
w2v_model.wv.most_similar('voldemort')

[('lord', 0.7069865465164185),
 ('wormtail', 0.6690964698791504),
 ('elder', 0.6340605616569519),
 ('nagini', 0.6284155249595642),
 ('pettigrew', 0.6125122904777527),
 ('understands', 0.611863911151886),
 ('weakness', 0.6112457513809204),
 ('dumbledore', 0.6090002655982971),
 ('quirrell', 0.6040014028549194),
 ('dumbledores', 0.6036220192909241)]

In [162]:
w2v_model.wv.most_similar('gryffindor')

[('ravenclaw', 0.72437983751297),
 ('slytherin', 0.7099413275718689),
 ('points', 0.698464572429657),
 ('hufflepuff', 0.6680050492286682),
 ('tower', 0.6656131148338318),
 ('spectators', 0.6427123546600342),
 ('locker', 0.6376153230667114),
 ('championship', 0.6364508867263794),
 ('chaser', 0.6340674161911011),
 ('captain', 0.6254349946975708)]

In [160]:
w2v_model.wv.most_similar('quidditch')

[('match', 0.7520051002502441),
 ('season', 0.7355700135231018),
 ('session', 0.6944305896759033),
 ('team', 0.6925305128097534),
 ('seeker', 0.6807789206504822),
 ('cup', 0.6791298985481262),
 ('player', 0.6632496118545532),
 ('winning', 0.6535742878913879),
 ('practice', 0.6534658670425415),
 ('wins', 0.6499863862991333)]

In [161]:
w2v_model.wv.most_similar('hermione')

[('ron', 0.654772162437439),
 ('parvati', 0.6493538618087769),
 ('encouragingly', 0.6356828808784485),
 ('hotly', 0.6309463977813721),
 ('unconvinced', 0.6253873109817505),
 ('incredulously', 0.624884843826294),
 ('doubtfully', 0.6231666803359985),
 ('lavender', 0.6096546649932861),
 ('grumpily', 0.6076415181159973),
 ('fearfully', 0.6051589250564575)]

Se detectan palabras "intrusas" utilizando el método:

```python
w2v_model.wv.doesnt_match(lista_de_palabras)
```

Este sirve para detectar cuál de las palabras no encaja semánticamente con el resto. Internamente, Word2Vec calcula el vector promedio de todas las palabras y devuelve aquella cuyo vector está más alejado del centroide de los demás.

In [164]:
w2v_model.wv.doesnt_match(['harry', 'ron', 'hermione', 'voldemort'])

'voldemort'

In [165]:
w2v_model.wv.doesnt_match(['malfoy', 'riddle', 'lestrange', 'weasley'])


'weasley'

#### Semántica relacional mediante analogías

Una de las capacidades más interesantes de los embeddings entrenados con Word2Vec es su habilidad para capturar relaciones semánticas vectoriales mediante operaciones aritméticas simples. Se utiliza el método:

```python
w2v_model.wv.most_similar(positive=[...], negative=[...])
```

Este permite construir este tipo de analogías mediante sumas y restas de vectores. Internamente, calcula:

$$
\text{resultado} = \sum \text{(positivos)} - \sum \text{(negativos)}
$$

y devuelve las palabras más similares al vector resultante, usando cosine similarity.

In [168]:
w2v_model.wv.most_similar(positive=["ron", "goyle"], negative=["harry"])


[('crabbe', 0.7475333213806152),
 ('pansy', 0.5595075488090515),
 ('parkinson', 0.52581387758255),
 ('guffawed', 0.504027247428894),
 ('cronies', 0.48564016819000244),
 ('george', 0.48499131202697754),
 ('zabini', 0.4760032296180725),
 ('sniggering', 0.4485675096511841),
 ('fred', 0.44503194093704224),
 ('gregory', 0.444950670003891)]